In [ ]:

import os
from pathlib import Path

# Set the working directory to the location of the notebook
notebook_dir = Path(os.getcwd()) / "02-experiment tracking"
os.chdir(notebook_dir)

print(os.getcwd())

/workspaces/mlops-zoomcamp/02-experiment tracking


## Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [9]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# client.list_
# return list of experiments
experiments = client.search_experiments()

for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}")

Name: my-cool-experiment, ID: 2
Name: nyc-taxi-experiment, ID: 1
Name: Default, ID: 0


In [8]:
# to create a new experiment
# client.create_experiment(name="my-cool-experiment")


from mlflow.entities import ViewType

# it returns runs fullfilling some contitions
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6", # only returns runs having lower values
    run_view_type=ViewType.ACTIVE_ONLY, # only active runs (not deleted),  DELETED_ONLY: only deleted runs,  ALL: both active and deleted
    max_results=5,
    order_by=["metrics.rmse ASC"] # order by rmse
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: e8e51c0c101948eab0fda8ec7a035943, rmse: 5.1065
run id: 65a7096112cb41b693e557e8008cfa9c, rmse: 5.1126
run id: e951c87d711843b680b409c58fdaf82a, rmse: 5.1126
run id: 120ea98d1f4043baa2687410b5169389, rmse: 5.1195
run id: 1e26342b34314a85b459910b6e37e640, rmse: 5.1448


## Interacting with the Model Registry
In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment nyc-taxi-regressor
2. Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
3. Transition the version 4 to "Staging" and adding annotations to it.

In [10]:
import mlflow

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
model_name = "nyc-taxi-regressor"

# register the best xgboost model
run_id = "65a7096112cb41b693e557e8008cfa9c" # 
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)

# register the best gradient boosting model
run_id = "4aa739211db94627b1744e36f4ee9664" 
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747334183102, current_stage='None', description=None, last_updated_timestamp=1747334183102, name='nyc-taxi-regressor', run_id='4aa739211db94627b1744e36f4ee9664', run_link=None, source=('/workspaces/mlops-zoomcamp/02-experiment '
 'tracking/mlruns/1/4aa739211db94627b1744e36f4ee9664/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=2>

In [16]:
# to put a model in 'staging' stage
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_10400/1125554250.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747334183102, current_stage='Staging', description=None, last_updated_timestamp=1747334693934, name='nyc-taxi-regressor', run_id='4aa739211db94627b1744e36f4ee9664', run_link=None, source=('/workspaces/mlops-zoomcamp/02-experiment '
 'tracking/mlruns/1/4aa739211db94627b1744e36f4ee9664/artifacts/model'), status='READY', status_message=None, tags={'model': 'gradient boosting'}, user_id=None, version=2>

In [ ]:
# to see the latest versions of models (in each stage separately)
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging


/tmp/ipykernel_10400/2139987129.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [22]:
# to put a model in 'production' stage
model_version = 2
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)

/tmp/ipykernel_10400/545672577.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747334183102, current_stage='Production', description=None, last_updated_timestamp=1747334948466, name='nyc-taxi-regressor', run_id='4aa739211db94627b1744e36f4ee9664', run_link=None, source=('/workspaces/mlops-zoomcamp/02-experiment '
 'tracking/mlruns/1/4aa739211db94627b1744e36f4ee9664/artifacts/model'), status='READY', status_message=None, tags={'model': 'gradient boosting'}, user_id=None, version=2>

In [23]:
# to see the latest versions of models
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Production


/tmp/ipykernel_10400/2139987129.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


#### update description

The following updates the description of a specific model version in the MLflow Model Registry.

- It’s purely metadata — it doesn’t change the stage or performance of the model, but is useful for tracking actions and auditability.

In [24]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=2,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1747334183102, current_stage='Production', description='The model version 2 was transitioned to Production on 2025-05-15', last_updated_timestamp=1747335460608, name='nyc-taxi-regressor', run_id='4aa739211db94627b1744e36f4ee9664', run_link=None, source=('/workspaces/mlops-zoomcamp/02-experiment '
 'tracking/mlruns/1/4aa739211db94627b1744e36f4ee9664/artifacts/model'), status='READY', status_message=None, tags={'model': 'gradient boosting'}, user_id=None, version=2>